<p align="center">
  <img src="assets/hydra_rknn.png" />
</p>

In [1]:
# Instalando o scikit-surprise
!pip3 install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 18.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095390 sha256=eb0b8f5297515f7e14486cbcf0163edb6498b904b32a83eedbd9b69003d21da4
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
import pandas as pd
import numpy as np
from ast import literal_eval
from nltk.stem.snowball import SnowballStemmer
from functools import reduce 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

import warnings; warnings.simplefilter('ignore')

# **RECOMENDADOR BASEADO EM CONTEÚDO**

In [3]:
# Carregando o dataset movies_metadata
md_filmes = pd.read_csv('movies_metadata.csv')
# Exibindo metadados dos filmes
md_filmes

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [4]:
md_filmes['genres'] = md_filmes['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance (x, list) else [])

In [5]:
md_filmes['year'] = pd.to_datetime(md_filmes['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [6]:
md_filmes.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995


In [7]:
# Carregando dataset links_small
links_small = pd.read_csv('links_small.csv', dtype={'movieId':"Int64",'imdbId':"Int64",'tmdbId':"Int64"})
# Vai ser usado no KNN
ds_links = links_small
# Converte o tipo da coluna, filtra os não nulos e cria uma serie com os valores de tmdbId
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')
# Exibindo a serie 
links_small

0          862
1         8844
2        15602
3        31357
4        11862
         ...  
9120    402672
9121    315011
9122    391698
9123    137608
9124    410803
Name: tmdbId, Length: 9112, dtype: int64

In [8]:
# Essas linhas possuem dados maus formatados, por isso foram removidas,
# o rest_index é para remover a coluna de index que foi criada, pois a 
# mesma estava fora de ordem 
md_filmes = md_filmes.drop([19730, 29503, 35587])
# Exibindo metadados dos filmes
md_filmes

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[Drama, Family]",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0,NaT
45462,False,NaN,0,[Drama],NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0,2011
45463,False,NaN,0,"[Action, Drama, Thriller]",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0,2003
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0,1917


In [9]:
# Convete o tipo da coluna 
md_filmes['id'] = md_filmes['id'].astype('int')

In [10]:
# Convete o tipo da coluna popularidade para float
md_filmes['popularity'] = md_filmes['popularity'].astype('float')

In [11]:
# Para conseguir remover as duplicatas 
md_filmes['genres'] = md_filmes['genres'].astype('str')

In [12]:
# Ordenando coluna de popularidade
md_filmes = md_filmes.sort_values('popularity', ascending=False).drop_duplicates().sort_index()

In [13]:
# Removendo os duplicados que contém mesmo valore de id, imdb_id, title
md_filmes = md_filmes.drop_duplicates(subset=['id', 'imdb_id', 'title'])

In [14]:
# Convete o tipo da coluna genero para list
md_filmes['genres'] = md_filmes['genres'].apply(literal_eval)

In [15]:
# Carregando dados dos credits  
credits = pd.read_csv('credits.csv')
# Carregando dados dos keywords  
keywords = pd.read_csv('keywords.csv')

In [16]:
# Converte o tipo da coluna para int
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md_filmes['id'] = md_filmes['id'].astype('int')

In [17]:
md_filmes.shape

(45433, 25)

In [18]:
# Fazendo a junção dos Dataframes de filmes com creditos e palavras chave 
md_filmes = md_filmes.merge(credits, on='id')
md_filmes = md_filmes.merge(keywords, on='id')

In [19]:
# Para conseguir remover as duplicatas 
md_filmes['genres'] = md_filmes['genres'].astype('str')

In [20]:
# Removendo duplicados
md_filmes = md_filmes.drop_duplicates()

In [21]:
md_filmes

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,status,tagline,title,video,vote_average,vote_count,year,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"['Animation', 'Comedy', 'Family']",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Released,NaN,Toy Story,False,7.7,5415.0,1995,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"['Adventure', 'Fantasy', 'Family']",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"['Romance', 'Comedy']",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,False,NaN,16000000,"['Comedy', 'Drama', 'Romance']",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,['Comedy'],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46491,False,NaN,0,"['Drama', 'Family']",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0,NaT,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...","[{'id': 10703, 'name': 'tragic love'}]"
46492,False,NaN,0,['Drama'],NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,Released,NaN,Century of Birthing,False,9.0,3.0,2011,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...","[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
46493,False,NaN,0,"['Action', 'Drama', 'Thriller']",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,Released,A deadly game of wits.,Betrayal,False,3.8,6.0,2003,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",[]
46494,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,Released,NaN,Satan Triumphant,False,0.0,0.0,1917,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",[]


In [22]:
# Convete o tipo da coluna genero para list
md_filmes['genres'] = md_filmes['genres'].apply(literal_eval)

In [23]:
# Colete todos os metadados para filmes no conjunto de dados links_small
# Filtre os filmes cujo id está presente no arquivo links.csv e armazene em smd
# .isin -> Se cada elemento no DataFrame está contido em valores.
# Similar movie data - smd
smd = md_filmes[md_filmes['id'].isin(links_small)]
# ds_filmes vai ser usado no KNN
ds_filmes = smd
smd.shape

(9086, 28)

In [24]:
smd['cast'] = smd['cast'].apply(literal_eval)

In [25]:
smd['crew'] = smd['crew'].apply(literal_eval)

In [26]:
smd['keywords'] = smd['keywords'].apply(literal_eval)

In [27]:
# Cria nova coluna com tamanho do cast
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))

In [28]:
# Cria nova coluna com tamanho do crew
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [29]:
smd.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,title,video,vote_average,vote_count,year,cast,crew,keywords,cast_size,crew_size
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Toy Story,False,7.7,5415.0,1995,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",13,106
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Jumanji,False,6.9,2413.0,1995,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1...",26,16
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,Grumpier Old Men,False,6.5,92.0,1995,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392...",7,4
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,Waiting to Exhale,False,6.1,34.0,1995,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':...",10,10
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,Father of the Bride Part II,False,5.7,173.0,1995,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...",12,7


In [30]:
def obter_diretor(x):
  for i in x:
    if i['job'] == 'Director':
      return i['name']
  return np.nan

In [31]:
smd['director'] = smd['crew'].apply(obter_diretor)

In [32]:
# Organiza o nome do cast
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [33]:
# Seleciona os 3 principais atores 
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [34]:
# Organiza as palavras chaves numa lista
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else []) 

In [35]:
# Remove os espaços e deixa tudo minusculo no cast
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [36]:
# Converte o tipo da coluna do director para string, pega o nome do diretor deixa minusculo e sem espaços
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))

In [37]:
# Repete 3 vezes o nome do diretor na lista para dar mais peso em relação a todo elenco
smd['director'] = smd['director'].apply(lambda x: [x,x, x])

In [38]:
# Criar uma serie palavras chave aplicando ao longo do axis=1 coluna
# stack ->retornar um dataframe remodelado, se as colunas tiverem um único nível, a saída é uma Série;
# Reseta index em 1 nível
palavras_chave = smd.apply(lambda x: pd.Series(x['keywords']), axis=1).stack().reset_index(level=1, drop=True)

In [39]:
# Nomeia a serie
palavras_chave.name = 'keyword'

In [40]:
# Retorna uma série contendo contagens de valores exclusivos.
palavras_chave = palavras_chave.value_counts()

#Exibindo as 5 palavras que ocorrem com mais frequencia  
palavras_chave[:5]

independent film        603
woman director          541
murder                  397
duringcreditsstinger    327
based on novel          310
Name: keyword, dtype: int64

In [41]:
# Selecionando somente as palavras que ocorrem mais de uma vez
palavras_chave = palavras_chave[palavras_chave > 1]

In [42]:
# Converterndo palavras em seu radical
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [43]:
def filtrar_palavras_chave(x):
  words = []
  for i in x:
    if i in palavras_chave:
      words.append(i)
  return words

In [44]:
# Aplicando a função filtrar palavras chaves
smd['keywords'] = smd['keywords'].apply(filtrar_palavras_chave)

In [45]:
# Aplica a lisa de palavras o stemmer que deixa somente o radical
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])

In [46]:
# Deixa as palavras minusculas e remove os espaços em palavras compostas 
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [47]:
# Criando sopa de palavras unindo as colunas 
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']

In [48]:
# Une as palavras da lista separando por espaços
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

In [49]:
# Ele é usado para transformar um determinado texto em um vetor com base na frequência (contagem) 
# de cada palavra que ocorre em todo o texto.
count = CountVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
# fit_transform -> Aprenda o dicionário de vocabulário e retorne a matriz documento-termo.
#       Isso é equivalente a fit seguido de transformação, mas implementado de forma mais eficiente.
count_matrix = count.fit_transform(smd['soup'])

In [50]:
# Calcule a similaridade de cosseno entre as amostras em X e Y.
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [51]:
# Cria uma coluna index contendo os indices do df - reset_index() -> Redefina o índice ou um nível dele.
smd = smd.reset_index()
# Cria uma serie com os titulos 
titles = smd['title']
# Faço o mapa reverso onde o id é o titulo e os valores são o id do filme
# Series -> Array unidimensional com rótulos de eixo (incluindo séries temporais).
indices = pd.Series(smd.index, index=smd['title'])

In [52]:
indices

title
Toy Story                                                0
Jumanji                                                  1
Grumpier Old Men                                         2
Waiting to Exhale                                        3
Father of the Bride Part II                              4
                                                      ... 
Rustom                                                9081
Mohenjo Daro                                          9082
Shin Godzilla                                         9083
The Beatles: Eight Days a Week - The Touring Years    9084
Force Majeure                                         9085
Length: 9086, dtype: int64

#**FILTRAGEM COLABORATIVA**

In [53]:
reader = Reader()

In [54]:
avaliacoes = pd.read_csv('ratings_small.csv')
# ds_avaliacoes vai ser usado no KNN
ds_avaliacoes = avaliacoes
avaliacoes.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [55]:
dados = Dataset.load_from_df(avaliacoes[['userId', 'movieId', 'rating']], reader)

In [56]:
svd = SVD()
cross_validate(svd, dados, measures=['RMSE', 'MAE'], verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8995  0.8957  0.8970  0.9007  0.8890  0.8964  0.0041  
MAE (testset)     0.6906  0.6904  0.6905  0.6951  0.6862  0.6906  0.0028  
Fit time          2.31    1.77    1.45    1.52    1.56    1.72    0.31    
Test time         0.16    0.15    0.14    0.16    0.23    0.17    0.03    


{'test_rmse': array([0.8994901 , 0.89571748, 0.89701849, 0.90068272, 0.8889508 ]),
 'test_mae': array([0.69062779, 0.69036293, 0.69045032, 0.6951367 , 0.68620613]),
 'fit_time': (2.3084354400634766,
  1.7725815773010254,
  1.450927495956421,
  1.5192410945892334,
  1.558323621749878),
 'test_time': (0.15846943855285645,
  0.1518266201019287,
  0.14370274543762207,
  0.1581108570098877,
  0.23070025444030762)}

In [57]:
trainset = dados.build_full_trainset()
svd.fit(trainset)

# **FUNÇÕES UTILITARIAS**

In [58]:
# Método que normaliza valores entre 0 e 1
# Recebe uma coluna como serie
def normalizacao_valores(serie):
  serie_index = serie.index
  array = serie.values
  y = array.reshape(-1, 1)
  scaler = MinMaxScaler(feature_range= (0, 1))
  rescaled = scaler.fit_transform(y)
  array = rescaled
  serie_normalizada = pd.DataFrame(array, index=serie_index)
  return serie_normalizada

In [59]:
# Aqui as vezes retorna um int e as vezes retorna uma serie tratei dessa forma
def get_index(title):
  # Df indices contem o titulo e o código do filme ao lado
  idx = indices[title]

  if isinstance(idx, pd.Series):
    return idx[0]

  return idx

# **RECOMENDADOR KNN**  

***Esse algoritmo utiliza a estratégia de filtragem colaborativa***  
Existem dois tipos:
* Baseado em itens;   
* Baseado em Usuários.  

***No nosso caso iremos utilizar a filtragem colaborativa baseado em usuários***

In [60]:
def recomendacao_KNN(usuarioId, ds_filmes=ds_filmes, ds_links=ds_links, ds_avaliacoes=ds_avaliacoes):

  # Verificando imdbids nulos
  ds_filmes[ds_filmes['imdb_id'].isnull()]

  # Verificando ids nulos
  ds_filmes[ds_filmes['id'].isnull()]

  # Extraindo coluna imdbId para pegar o código do filme em links
  ds_filmes['imdbId'] = ds_filmes['imdb_id'].apply(lambda x: x if pd.isna(x) else str(x)[2:])

  # Alterando tipo da coluna
  ds_filmes['imdbId'] = ds_filmes['imdbId'].astype('int')

  # Merge pegando os movieIds de Links
  ds_filmes = pd.merge(ds_filmes, ds_links, on='imdbId')

  # Removendo a coluna tmdbId
  ds_filmes.drop(['tmdbId'], axis=1, inplace=True)
  
  # Para fazer a filtragem colaborativa o que me interessa é o Id do Filme e o titulo
  # Selecionando/filtrandos as colunas que irei utilizar 
  ds_filmes = ds_filmes[['movieId', 'title']]

  # Filtrando as colunas que preciso
  ds_avaliacoes = ds_avaliacoes[['userId', 'movieId', 'rating']]
  
  # Juntar as duas bases, semelhante ao join no SQL 
  df_filmes_avaliacoes = pd.merge(ds_filmes, ds_avaliacoes, on='movieId')
  

  # Pega o id do usuário e coloca como indice da coluna e pivotar a tabela 
  # Onde os titulos apareçam em cima e as avaliações serão os valores 
  df_recommender = df_filmes_avaliacoes.pivot_table(index='userId', columns='title', values='rating').fillna(0)
  

  # Modelo dos vizinhos próximos - KNN
  # Distância do cosseno não sofre muito com a maldição da dimensionalidade
  # Instancia o modelo KNN, usando a metrica do cosseno(hiperparametros)
  modelo_knn = NearestNeighbors(metric='cosine')
  # Calcula todas as distâncias em relação aos vizinhos próximos
  modelo_knn.fit(df_recommender)

  # Quantidade de vizinhos mais próximos que quero
  qtde_vizinhos = 4

  # Recupera a linha que o usuário se encontra
  # Usuario que eu quero recomendar algo
  idx_usuario_knn = df_recommender.index.get_loc(df_recommender.loc[usuarioId].name)
  # Quando chamar o modelo de KNN preciso saber a distância dos vizinhos próximos
  # E quais são os indices vizinhos próximos
  # Irei passar no parâmetro de kneighbors() o registro que quero usar para puxar
  # Vizinho mais próximo - Fiz uma mudança ao invés de iloc usei loc
  distancia_vizinhos, indices_vizinhos = modelo_knn.kneighbors(df_recommender.iloc[idx_usuario_knn].values.reshape(1, -1), n_neighbors=qtde_vizinhos)

  usuario = df_recommender.index[idx_usuario_knn]

  # Cria a lista que irá conter os dataframes
  dfs_vizinhos_proximo = []

  # For para percorrer cada um da lista 
  # Usei o flatter para remover uma dimensão 
  for i in range(0, len(distancia_vizinhos.flatten())):
    # Pula o primeiro elemento pois vai ser ele mesmo
    if i == 0:
      ds_usuario_knn = df_recommender.loc[usuario].to_frame()
      dfs_vizinhos_proximo.append(ds_usuario_knn)
    else:
      # Localiza o vizinho mais próximo em df_recommender
      vizinho_proximo = df_recommender.index[indices_vizinhos.flatten()[i]]
      # Adiciona o vizinho próximo a lista dos dataframes
      dfs_vizinhos_proximo.append(df_recommender.loc[vizinho_proximo].to_frame())
  

  # Pegando o indice dos vizinhos mais próximos
  idx_vizinhos_prox = [col for i in range(len(dfs_vizinhos_proximo)) for col in dfs_vizinhos_proximo[i] if i != 0]
  
  # Faz o merge da dataframe do usuário com seus vizinhos
  ds_titulos = reduce(lambda df_esq, df_dir: pd.merge(df_esq, df_dir, on=['title']), dfs_vizinhos_proximo)
  
  # Ordena de forma descrescente pelos vizinhos mais próximos
  ds_titulos = ds_titulos.sort_values(by=idx_vizinhos_prox, ascending=False)
  
  # Filtrar todos os titulos do vizinho que foram > 0 e onde usuário não assistiu
  ds_titulos = ds_titulos[((ds_titulos[idx_vizinhos_prox[0]] > 0) | (ds_titulos[idx_vizinhos_prox[1]] > 0) | (ds_titulos[idx_vizinhos_prox[2]] > 0)) & (ds_titulos[usuario] == 0)]
  
  # Criando e calculando a média 
  ds_titulos['Media'] = (ds_titulos[idx_vizinhos_prox[0]] + ds_titulos[idx_vizinhos_prox[1]] + ds_titulos[idx_vizinhos_prox[2]]) / len(idx_vizinhos_prox)
  
  # Resetando o indice
  ds_titulos.reset_index(inplace=True)
  
  # Atribui a média os valores de média normalizados 
  ds_titulos['est'] = normalizacao_valores(ds_titulos['Media'])

  # Filtrando as colunas que irei devolver ao método híbrido 
  titulos_recomendacao_KNN = ds_titulos[['title', 'est']]
  titulos_recomendacao_KNN = titulos_recomendacao_KNN.sort_values(by=['est'], ascending=False)


  return titulos_recomendacao_KNN.head(10)


#**RECOMENDADOR HÍBRIDO - HYDRA**

In [61]:
def convert_int(x):
  try:
    return int(x)
  except:
    return np.nan

In [62]:
# Carregando o arquivo que contem os ids dos filmes, selecionando 
# as colunas 'movieId', 'tmdbId'
id_map = pd.read_csv('links_small.csv')[['movieId', 'tmdbId']]
# Converte a coluna id_map['tmdbId'] para inteiro
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
# Mudando os nomes da coluna tmdbId para id
id_map.columns = ['movieId', 'id']
# Fazendo o merge de id_map com SMD
id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')

In [63]:
indices_map = id_map.set_index('id')

In [64]:
# Junta e soma as estimativas de cada dataframe 
def agrupa_recomendacoes(dataframe1, dataframe2):
  dfs_concatenados = pd.concat([dataframe1, dataframe2], ignore_index=True)
  dfs_agrupados = dfs_concatenados.groupby('title')[['est']].mean()
  # dfs_agrupados = dfs_concatenados.groupby('title')['est'].sum()

  dfs_agrupados = dfs_agrupados.sort_values('est', ascending=False)
  dfs_agrupados = dfs_agrupados.reset_index()

  return dfs_agrupados


In [65]:
def recomendador_hydra(userId, title):

  if isinstance(userId, int) and isinstance(title, str):

    if (userId in avaliacoes['userId'].values) and (title in id_map.index) :
      
      idx = get_index(title)

      titulos_recomendados_KNN = recomendacao_KNN(userId)

      sim_scores = list(enumerate(cosine_sim[int(idx)]))
      sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
      sim_scores = sim_scores[1:26]
      movie_indices = [i[0] for i in sim_scores]

      movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
      movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
      movies['est'] = normalizacao_valores(movies['est'])
      movies = movies[['title', 'est']]

      movies = movies.sort_values('est', ascending=False)

      movies = movies.head(10)

      recomendacao_hibrida = agrupa_recomendacoes(titulos_recomendados_KNN, movies)

      return recomendacao_hibrida
    
    else:
      print('Erro: O id do usuário ou nome do filme não consta no dataset')
  else:
    print('Algo inesperado aconteceu. Tente novamente!')

In [73]:
recomendador_hydra(1, "Avatar")

,title,est
0,Beverly Hills Cop,1.000000
1,The Terminator,1.000000
2,X-Men: Days of Future Past,0.917433
3,Terminator 2: Judgment Day,0.869700
4,Star Trek Into Darkness,0.831581
5,Fantastic Planet,0.828785
6,Aliens,0.772969
7,Sinbad and the Eye of the Tiger,0.692656
8,Titus,0.642857
9,Proof,0.642857


In [74]:
recomendador_hydra(500, 'Avatar')

,title,est
0,Walk the Line,1.000000
1,Aliens,1.000000
2,"I, Robot",0.920000
3,Apollo 13,0.920000
4,X-Men: Days of Future Past,0.898953
5,Rain Man,0.880000
6,Spider-Man,0.880000
7,Being John Malkovich,0.880000
8,Fantastic Planet,0.871124
9,Sinbad and the Eye of the Tiger,0.840110
